In [53]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnistt
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [55]:
# SETTING UP HYPERPARAMETERS AND DATA SET PARAMETERS
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
# Training parameters.

learning_rate = 0.01
training_steps = 1000
batch_size = 50
display_step = 100

# LOADING AND PREPARING THE MNIST DATA SET
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize images value from [0, 255] to [0, 1].

x_train, x_test = x_train / 255., x_test / 255.
print(len(x_train),len(x_test),len(y_train),len(y_test))

# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

#prefetch: Creates a `Dataset` that prefetches elements from this dataset. 
#Like other `Dataset` methods, prefetch operates on the #elements of the input dataset. 
#It has no concept of examples vs. batches. `examples.prefetch(2)` will prefetch 
#two elements (2 examples), #while `examples.batch(20).prefetch(2)` will prefetch 
#2 elements (2 batches, of 20 examples each).
train_data=train_data.shuffle(x_train.shape[0]).repeat().batch(batch_size).prefetch(1)

60000 10000 60000 10000


In [56]:
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [57]:
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [72]:
W_conv1 = weight_variable([3, 3, 1, 16])
b_conv1 = bias_variable([16])

W_conv2 = weight_variable([3, 3, 16, 8])
b_conv2 = bias_variable([8])

W_fc1 = weight_variable([7 * 7 * 8, 64])
b_fc1 = bias_variable([64])

W_fc2 = weight_variable([64, 10])
b_fc2 = bias_variable([10])

In [75]:
def deepnn(x,keep_prob):
    # Reshape to use within a convolutional neural net.
    # Last dimension is for "features" - there is only one here, since images are
    # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # First convolutional layer - maps one grayscale image to 32 feature maps.
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    # Pooling layer - downsamples by 2X.
    h_pool1 = max_pool_2x2(h_conv1)
    # Second convolutional layer -- maps 32 feature maps to 64.
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    # Second pooling layer.
    h_pool2 = max_pool_2x2(h_conv2)
    # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
    # is down to 7x7x64 feature maps -- maps this to 1024 features.
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*8])
    
        
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    
    # Map the 1024 features to 10 classes, one for each digit
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return tf.nn.softmax(y_conv)

In [76]:
# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(1e-4)


# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# DEFINING OPTIMIZERS AND ACCURACY METRICS
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# OPTIMIZATION PROCESS AND UPDATING WEIGHTS AND BIASES
def run_optimization(x, y):
# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = deepnn(x,0.5)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W_conv1, W_conv2, W_fc1, W_fc2,b_conv1, b_conv2, b_fc1, b_fc2])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W_conv1, W_conv2, W_fc1, W_fc2,b_conv1, b_conv2, b_fc1, b_fc2]))
    
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = deepnn(batch_x,0.0)
        #print(pred)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    if step % 100 ==0:
        print("Test Accuracy: %f" % accuracy(deepnn(x_test,0.0), y_test))
        
# Test model on test set.
pred = deepnn(x_test,0.0)
print("Test Accuracy: %f" % accuracy(pred, y_test))

step: 100, loss: 107.469070, accuracy: 0.340000
Test Accuracy: 0.392300
step: 200, loss: 95.586945, accuracy: 0.480000
Test Accuracy: 0.503500
step: 300, loss: 72.679291, accuracy: 0.520000
Test Accuracy: 0.600600
step: 400, loss: 61.386925, accuracy: 0.640000
Test Accuracy: 0.652300
step: 500, loss: 50.518440, accuracy: 0.760000
Test Accuracy: 0.684600
step: 600, loss: 47.948830, accuracy: 0.720000
Test Accuracy: 0.700900
step: 700, loss: 33.211571, accuracy: 0.820000
Test Accuracy: 0.700600
step: 800, loss: 40.263382, accuracy: 0.760000
Test Accuracy: 0.708400
step: 900, loss: 39.737679, accuracy: 0.760000
Test Accuracy: 0.719300
step: 1000, loss: 45.920574, accuracy: 0.640000
Test Accuracy: 0.723100
Test Accuracy: 0.723100


In [71]:
%tensorboard --logdir logs/

UsageError: Line magic function `%tensorboard` not found.
